In [2]:
import nltk
import re                                  # library for regular expression operations
import string 
# nltk.download('punkt')
# nltk.download('wordnet')
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
import pandas as pd
from rake_nltk import Rake
from nltk.stem import WordNetLemmatizer

In [3]:
# wordnet_lemmatizer = WordNetLemmatizer()

In [4]:
# from http://www.lextek.com/manuals/onix/stopwords1.html
stopwords = set(w.rstrip() for w in open('stopwords.txt'))

In [5]:
df= pd.read_csv('C:\\Users\\oazed\\TWITTER\\twint\\TWEET001.csv')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
df = df[df['language'] == 'en']

In [8]:
df = df[[ 'tweet']]

In [9]:
df.shape

(47230, 1)

In [10]:
# tweet = tweet.sort_values(by = "likes_count", ascending=False).reset_index()

In [11]:
# tweet.to_csv('TWEET.csv',index=False)

In [12]:
df.head()

,tweet
0,"Omicron fears slumped, Investors returning wit..."
2,"US Dollar Price Action Setups: EUR/USD, GBP/US..."
5,$USD $DXY US Dollar Pre-NFP Price Action Setu...
6,We also sold $AUDUSD and bought $USDCAD in Asi...
16,"Wednesday January 5, 2022 The Daily Table: a ..."


In [15]:
df['key_words'] = ""

In [52]:
for index, row in df.iterrows():
    tweet = row['tweet']
#     tweet = tweet.lower()
    numbers = re.sub(r'[0-9]','', tweet)
    tags = re.sub(r'#', '', numbers)
    tokens = re.sub(r'https?:\/\/.*[\r\n]*', '', tags)
    special = re.sub('[^a-zA-Z0-9 \n\.]','',tokens)
#     tokens = re.sub(r'/\$\w*', '', tweet)
#     tokens = re.sub(r'^RT[\s]+', '', tweet)
#     tokens = re.sub(r'#', '', tweet)
#     tokens = nltk.tokenize.word_tokenize(tweet)
#     tokens = [t for t in tokens if len(t) > 2]
#     tokens = [t for t in tokens if t not in stopwords]
    r = Rake()
    r.extract_keywords_from_text(special)
    keyword_dict_scores = r.get_word_degrees()
#     row['key_words']= special
    row['key_words']= list(keyword_dict_scores.keys())

In [53]:
df.head()

,tweet,key_words
0,"Omicron fears slumped, Investors returning wit...","[omicron, fears, slumped, investors, returning..."
2,"US Dollar Price Action Setups: EUR/USD, GBP/US...","[us, dollar, price, action, setups, eurusd, gb..."
5,$USD $DXY US Dollar Pre-NFP Price Action Setu...,"[usd, dxy, us, dollar, prenfp, price, action, ..."
6,We also sold $AUDUSD and bought $USDCAD in Asi...,"[also, sold, audusd, bought, usdcad, asia, wok..."
16,"Wednesday January 5, 2022 The Daily Table: a ...","[wednesday, january, daily, table, wrap, overn..."


In [61]:
df2 = df[['key_words']].reset_index()

In [62]:
df2 = df2[['key_words']]

In [70]:
df2.shape

(47230, 2)

In [66]:
df2['BoW'] = ""
columns = df2.columns
for index, row in df2.iterrows():
    words = ''
    for col in columns:
        words = words +' '.join(row[col])+ ' '
    row['BoW'] = words

In [67]:
df2.head()

,key_words,BoW
0,"[omicron, fears, slumped, investors, returning...",omicron fears slumped investors returning inve...
1,"[us, dollar, price, action, setups, eurusd, gb...",us dollar price action setups eurusd gbpusd au...
2,"[usd, dxy, us, dollar, prenfp, price, action, ...",usd dxy us dollar prenfp price action setups e...
3,"[also, sold, audusd, bought, usdcad, asia, wok...",also sold audusd bought usdcad asia woke targe...
4,"[wednesday, january, daily, table, wrap, overn...",wednesday january daily table wrap overnight m...


In [69]:
df2.to_csv("checktwt.csv",index=False)

In [101]:
tweet_text = df2['BoW'].tolist()


In [102]:
tweet_text[0]

'omicron fears slumped investors returning invest  '

In [103]:
def my_tokenizer(s):
#     s = s.lower() # downcase
#     # remove stock market tickers like $GE
#     tokens = re.sub(r'/\$\w*', '', s)
#     # remove old style retweet text "RT"
#     tokens = re.sub(r'^RT[\s]+', '', s)
#     # remove hyperlinks
#     tokens = re.sub(r'https?:\/\/.*[\r\n]*', '', s)
#     # remove hashtags
#     # only removing the hash # sign from the word
#     tokens = re.sub(r'#', '', s)
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
#     tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
#     tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
#     tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    return tokens

In [114]:
word_index_map = {}
current_index = 0
orig_tweets = []

In [121]:
tweet_text

['omicron fears slumped investors returning invest  ',
 'us dollar price action setups eurusd gbpusd audusd usdcad usd dxy  ',
 'usd dxy us dollar prenfp price action setups eurusd gbpusd audusd  ',
 'also sold audusd bought usdcad asia woke targets funda techs us yields fed min hawkish nfp expected aud rba dovish cad jobs weak new covid restrictions zip sell buy  ',
 'wednesday january daily table wrap overnight markets ausbiz ausecon bonds rates fx audusd aud usdaud metals lme metalprices commsec  ',
 'friday january daily table wrap overnight markets ausbiz ausecon bonds rates fx audusd aud usdaud metals lme metalprices commsec  ',
 'thursday january daily table wrap overnight markets ausbiz ausecon bonds rates fx audusd aud usdaud metals lme metalprices commsec  ',
 'relative strength numbers day looking buying opportunities using strong currencies gbpchfcad weak jpynzdeur swing trading time frames h relativestrength fx audusd usdcad usdchf eurusd gbpusd nzdusd usdjpy  ',
 'austral

In [127]:
for tweets in tweet_text:
    token = my_tokenizer(tweets)
    orig_tweets.append(token)
# cleaned = [item for item in orig_tweets if not isinstance(item,list)]

    for t in token:
        if t not in word_index_map:
            word_index_map[t] = current_index
            current_index += 1


In [128]:
word_index_map

{'omicron': 0,
 'fears': 1,
 'slumped': 2,
 'investors': 3,
 'returning': 4,
 'invest': 5,
 'dollar': 6,
 'price': 7,
 'action': 8,
 'setups': 9,
 'eurusd': 10,
 'gbpusd': 11,
 'audusd': 12,
 'usdcad': 13,
 'usd': 14,
 'dxy': 15,
 'prenfp': 16,
 'sold': 17,
 'bought': 18,
 'asia': 19,
 'woke': 20,
 'targets': 21,
 'funda': 22,
 'techs': 23,
 'yields': 24,
 'fed': 25,
 'min': 26,
 'hawkish': 27,
 'nfp': 28,
 'expected': 29,
 'aud': 30,
 'rba': 31,
 'dovish': 32,
 'cad': 33,
 'jobs': 34,
 'weak': 35,
 'covid': 36,
 'restrictions': 37,
 'zip': 38,
 'sell': 39,
 'buy': 40,
 'wednesday': 41,
 'january': 42,
 'daily': 43,
 'table': 44,
 'wrap': 45,
 'overnight': 46,
 'markets': 47,
 'ausbiz': 48,
 'ausecon': 49,
 'bonds': 50,
 'rates': 51,
 'usdaud': 52,
 'metals': 53,
 'lme': 54,
 'metalprices': 55,
 'commsec': 56,
 'friday': 57,
 'thursday': 58,
 'relative': 59,
 'strength': 60,
 'day': 61,
 'looking': 62,
 'buying': 63,
 'opportunities': 64,
 'using': 65,
 'strong': 66,
 'currencies': 67,

In [129]:
data_items = word_index_map.items()
data_list = list(data_items)
df = pd.DataFrame(data_list)


In [130]:
words =df.rename(columns={0: 'Word',1:'Frequency'})

In [133]:
words.to_csv('WORDS.csv',index=False)

In [76]:
idx = df[df['Word'].apply(lambda x: x.startswith("/"))]

In [77]:
df = df.drop(idx.index).reset_index()

In [78]:
# df.to_csv('Words.csv',index=False)

In [79]:
df = df[~df.Word.str.contains(r'[0-9]')]

In [80]:
df.shape

(28744, 3)

In [81]:
df = df[~df.Word.str.contains(r'[^0-9a-zA-Z]')]

In [82]:
df.shape

(24835, 3)

In [83]:
df.Word.unique()

array(['announce', 'ive', 'aped', ..., 'niggas', 'pablosabbatella',
       'vale'], dtype=object)

In [87]:
df = df.sort_values(by = "Frequency", ascending=False).reset_index()

In [88]:
#  df.to_csv('Frequent_Words.csv',index=False)

In [90]:
df = df[['Word','Frequency']]

In [91]:
df.head()

,Word,Frequency
0,vale,61112
1,pablosabbatella,61111
2,niggas,61110
3,unrulybtch,61109
4,fedex,61107


In [92]:
df.shape

(24835, 2)

In [93]:
24835/2

12417.5

In [96]:
df['Word_2'] = df.Word[12417:]

<ipython-input-96-34dd09571394>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Word_2'] = df.Word[12417:]


In [98]:
df['Word_1'] = df.Word[:12417]

In [100]:
df2 = df[['Word_1','Word_2']]

In [102]:
df2

,Word_1,Word_2
0,vale,NaN
1,pablosabbatella,NaN
2,niggas,NaN
3,unrulybtch,NaN
4,fedex,NaN
...,...,...
24830,NaN,boredapeyc
24831,NaN,bayc
24832,NaN,aped
24833,NaN,ive
